# Flask Server Example
There are a few considerations when using AgentOps in a server setting.

AgentOps uses a [Singleton](https://refactoring.guru/design-patterns/singleton) pattern. This means that you can call functions on the agentops module and the client is inferred because only one client exists at a time.

Ex:

```python
import agentops

agentops.init()  # <-- This creates a default client that the entire module uses
agentops.end_session(
    end_state="Success"
)  # <-- functions are called on the default client
```

In simple use-cases, each agentops client only needs to maintain one [Session](https://docs.agentops.ai/v1/concepts/core-concepts#sessions). In a server setting, its likely that the developer wants to be able to handle multiple sessions concurrently.

For this, we have exposed a function `agentops.start_session()` to create new sessions. This function allows for the AgentOps client to maintain multiple sessions at the same time. Once more than 1 session is started, however, `session_id: str` parameters become required in the following functions:
- `agentops.end_session()`
- `agentops.add_tags()`
- `agentops.set_tags()`
- `agentops.record()`

## Creating our Server

In [1]:
agentops = None  # removing the above instance
import agentops

from fastapi import FastAPI
import uvicorn
import threading
from dotenv import load_dotenv

load_dotenv()

agentops.init()  # the server_hosted param is not required, but it blocks the creation of a default Client and saves memory on the machine.

app = FastAPI()


### ---- code to run the app in jupyter notebook ----
class ServerThread(threading.Thread):

    def __init__(self):
        super(ServerThread, self).__init__()
        self.stop_event = threading.Event()

    def stop(self):
        # Stop the Flask server
        self.stop_event.set()

    def run(self):
        while not self.stop_event.is_set():
            uvicorn.run(app, host="0.0.0.0", port=9696)


### -------------------------------------------------

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=63e795a8-5e91-4ba9-aa4b-e0a52fd42c7d


## AgentOps Client
A lot of AgentOps is designed to be __"magical"__, but more complicated applications like use on an API server require us to pull back the curtain a bit.

In [2]:
import agentops
from agentops import ActionEvent
from openai import OpenAI

openai = OpenAI()

Instead of calling `agentops.init()` as we normally would, instead we'll use what's called a [Factory Pattern](https://refactoring.guru/design-patterns/factory-method). This allows us to create a client, and in-turn a session, as needed.

In [3]:
@app.get("/completion")
def completion():

    session_id = agentops.start_session(tags=["user-hello"])

    messages = [{"role": "user", "content": "Hello"}]
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.5,
        session_id=session_id,  # <-- add the agentops session_id to the create function
    )

    if "hello" in str(response.choices[0].message.content).lower():
        agentops.record(
            ActionEvent(
                action_type="Agent says hello",
                params=messages,
                returns=str(response.choices[0].message.content),
            ),
            session_id=session_id,  # <-- be sure to include session_id here if there are multiple sessions at once
        )

    agentops.end_session(end_state="Success", session_id=session_id)

    return {"response": response}

Let's start the server

In [4]:
# Running server in a different thread
server_thread = ServerThread()
server_thread.start()

we'll give the server a few seconds to start

In [5]:
import time

time.sleep(2)

INFO:     Started server process [53716]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9696 (Press CTRL+C to quit)


Now let's send a request to make our completion

In [6]:
import requests

response = requests.get("http://localhost:9696/completion")
response = requests.get("http://localhost:9696/completion")

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=8239934d-7cc2-4979-95d0-2382dad2a2be


INFO:     127.0.0.1:50287 - "GET /completion HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/uvicorn/protocols/http/httptools_impl.py", line 426, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/uvicorn/middleware/proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/opt/homebrew/lib/python3.11/site-packages/starlette/applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/opt/homebrew/lib/python3.11/site-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/opt/homebrew/lib/python3.11/site-packages/starlett

INFO:     127.0.0.1:50294 - "GET /completion HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/uvicorn/protocols/http/httptools_impl.py", line 426, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/uvicorn/middleware/proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/opt/homebrew/lib/python3.11/site-packages/starlette/applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/opt/homebrew/lib/python3.11/site-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/opt/homebrew/lib/python3.11/site-packages/starlett

Now if we look at the session printed in the output, we should see an LLM Event and Action Event

![ss](assets/fastapi-1.png)

In [7]:
server_thread.stop()